# Export to JSON pipeline

Load the data (previously created pandas dataframe exported as a json)

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./interactive/lol.csv')

In [3]:
data.head(5)

,contcod,ppp_2005,ppp_percentile,cum_perc
0,BRA,129.97,1.13,2.5
1,BRA,534.98,13.46,7.5
2,BRA,792.29,22.71,12.5
3,BRA,1022.31,29.78,17.5
4,BRA,1215.79,35.24,22.5


It's not visible, but the values have more than 2 decimal places

In [4]:
data.columns

Index(['contcod', 'ppp_2005', 'ppp_percentile', 'cum_perc'], dtype='object')

In [5]:
for column in data.columns[1:]:
  print(column)

ppp_2005
ppp_percentile
cum_perc


Round to 2 decimal places everything other than 1st column

In [6]:
for column in data.columns[1:]:
  data[column] = round(data[column], 2)

In [7]:
countries = data['contcod'].unique()

In [8]:
for country in countries:
  print(country)

BRA
CHN
DEU
IND


In [9]:
data.head(2)

,contcod,ppp_2005,ppp_percentile,cum_perc
0,BRA,129.97,1.13,2.5
1,BRA,534.98,13.46,7.5


Only select columns we'd like:

In [10]:
data = data[['contcod', 'cum_perc', 'ppp_percentile']]
data.head(2)

,contcod,cum_perc,ppp_percentile
0,BRA,2.5,1.13
1,BRA,7.5,13.46


Rename the columns for personal convenience:

In [11]:
data.columns = ['contcod', 'x', 'y']

In [12]:
data.head(2)

,contcod,x,y
0,BRA,2.5,1.13
1,BRA,7.5,13.46


Create custom `dict` for export that looks like this:


```
{
   'IND': 
       [
           {'x': 2.5, 'y': 25.3}, 
           {'x': 7.5, 'y': 31.5}, 
           ... 
       ],
   'DEU': [  ...  ],
   ...
}
```

In [13]:
dict_for_export = {}

In [14]:
for country in countries:
  
  temp_list = []
  
  def append_to_list(x, y):
    temp_list.append({'x': x, 'y': y})
  
  data[data['contcod'] == country][['x', 'y']].apply(lambda row: append_to_list(row['x'], row['y']), axis=1)
    
  dict_for_export[country] = temp_list

Preview contents:

In [15]:
dict_for_export['BRA'][:3]

[{'x': 2.5, 'y': 1.13}, {'x': 7.5, 'y': 13.46}, {'x': 12.5, 'y': 22.71}]

In [16]:
dict_for_export['DEU'][17:20]

[{'x': 87.5, 'y': 96.6}, {'x': 92.5, 'y': 97.67}, {'x': 97.5, 'y': 99.4}]

Save to file:

In [17]:
import json

In [18]:
output = json.dumps(dict_for_export)

In [19]:
with open("./interactive/PY_DATA.js", "w") as text_file:
    text_file.write("var DATA = " + output)